# Cluster

## Objectives

* Fit and evaluate a cluster model to group similar data
* Understand the profile for each cluster


## Inputs

* outputs/datasets/cleaned/FertilityTreatmentDataCleaned.csv
* Instructions on which variables to use for data cleaning and feature engineering, found in their respective notebooks.

## Outputs

* Cluster Pipeline
* Train Set
* Most important features to define a cluster plot
* Clusters Profile Description
* Cluster Silhouette


---

## Change working directory

Change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

To make the parent of the current directory the new current directory:
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("A new current directory has been set")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In [ ]:
import numpy as np
import pandas as pd

# Open dataset
df = pd.read_csv("outputs/datasets/cleaned/FertilityTreatmentDataCleaned.csv")
        
print(df.shape)
df.head(3)
df.columns


---

## ML Pipeline with all data

#### Cluster Preprocessing Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine.encoding import OneHotEncoder
from src.custom_transformers import OrdinalEncoderWithCategories
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


# Define the natural order for each feature
categories = [
    ['18-34', '35-37', '38-39', '40-42', '43-44', '45-50'],  # Patient age at treatment
    ['0', '1', '2', '3', '4', '5', '>5'],                    # Total number of previous IVF cycles
    ['18-34', '35-37', '38-39', '40-42', '43-44', '45-50'],  # Patient/Egg provider age
    ['18-34', '35-37', '38-39', '40-42', '43-44', '45-50', '51-55', '56-60', '>60'],  # Partner/Sperm provider age
    ['0', '0 - frozen cycle', '1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '>40'], # Fresh eggs collected
    ['0', '0 - frozen cycle', '1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '>40'], # Total eggs mixed
    ['0', '0 - frozen cycle', '1-5', '6-10', '11-15', '16-20', '21-25', '26-30', '>30'],  # Total embryos created
    ['0', '1', '1e', '2', '3'],  # Embryos transferred
    ['0 - fresh cycle', '0 - frozen cycle', '1-5', '6-10', '>10']  # Total embryos thawed
]

# Define columns that need ordinal encoding
ordinal_columns = [
    'Patient age at treatment',
    'Total number of previous IVF cycles',
    'Patient/Egg provider age',
    'Partner/Sperm provider age',
    'Fresh eggs collected',
    'Total eggs mixed',
    'Total embryos created',
    'Embryos transferred',
    'Total embryos thawed'
]

def PipelineFECluster():
    pipeline_base = Pipeline(
        [   
            ('ordinal_encoding', OrdinalEncoderWithCategories(
                categories=categories,
                columns=ordinal_columns
                )
            ),
            (
                "one_hot_encoding",
                OneHotEncoder(
                    variables=[
                        "Specific treatment type",
                        "Egg source",
                        "Sperm source",
                        "Patient ethnicity",
                        "Date of embryo transfer"
                    ],
                ),
            ),
            (
                "smart_correlation",
                SmartCorrelatedSelection(
                    method="spearman",
                    threshold=0.9,
                    selection_method="variance",
                ),
            ),
        ]
    )
    return pipeline_base

Clustering pipeline:

Pipeline for scaling, PCA and modelling

In [ ]:
def PipelineCluster():
    pipeline_base = Pipeline(
        [
        ("scaler", StandardScaler()),
            # Leave at 50 components for now
            ("pca", PCA(n_components=50, random_state=0)),
            # Leave at 50 clusters for now
            ("model", KMeans(n_clusters=50, random_state=0))
        ]
    )
    return pipeline_base

In [ ]:
pre_processing_cluster = PipelineFECluster()
df = pre_processing_cluster.fit_transform(df)


## Principal Component Analysis (PCA)

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_pca = Pipeline(pipeline_cluster.steps[:-2])
df_pca = pipeline_pca.fit_transform(df)

print(df_pca.shape,'\n', type(df_pca))

Apply PCA separately to the scaled data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
# Set the number of components to the number of features
n_components = 54


def pca_components_analysis(df_pca, n_components):
    pca = PCA(n_components=n_components).fit(df_pca)
    x_PCA = pca.transform(df_pca)  # array with transformed PCA

    ComponentsList = ["Component " + str(number) for number in range(n_components)]
    dfExplVarRatio = pd.DataFrame(
        data=np.round(100 * pca.explained_variance_ratio_, 3),
        index=ComponentsList,
        columns=["Explained Variance Ratio (%)"],
    )

    dfExplVarRatio["Accumulated Variance"] = dfExplVarRatio[
        "Explained Variance Ratio (%)"
    ].cumsum()

    PercentageOfDataExplained = dfExplVarRatio["Explained Variance Ratio (%)"].sum()

    print(
        f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n"
    )
    plt.figure(figsize=(9, 6))
    sns.lineplot(data=dfExplVarRatio, marker="o")
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0, 110, 10))
    plt.show()


pca_components_analysis(df_pca=df_pca, n_components=n_components)

In [ ]:
pca_components_analysis(df_pca=df_pca,n_components=27)

In [ ]:
def PipelineCluster():
    pipeline_base = Pipeline(
        [
        ("scaler", StandardScaler()),
            # Apply PCA with chosen number of components
            ("pca", PCA(n_components=27, random_state=0)),
            # Leave at 50 clusters for now
            ("model", KMeans(n_clusters=50, random_state=0))
        ]
    )
    return pipeline_base

## Elbow Method and Silhouette Score

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_analysis = Pipeline(pipeline_cluster.steps[:-1])
df_analysis = pipeline_analysis.fit_transform(df)
print(df_analysis.shape,'\n', type(df_analysis))

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11)) # 11 is not inclusive, it will plot until 10
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

# 8 is not inclusive, it will stop at 6
n_cluster_start, n_cluster_stop = 2, 7

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(
    KMeans(random_state=0), k=(n_cluster_start, n_cluster_stop), metric="silhouette"
)
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(
        estimator=KMeans(n_clusters=n_clusters, random_state=0), colors="yellowbrick"
    )
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")

## Fit Cluster Pipeline

In [ ]:
def PipelineCluster():
    pipeline_base = Pipeline(
        [
        ("scaler", StandardScaler()),
            # Apply PCA with chosen number of components
            ("pca", PCA(n_components=27, random_state=0)),
            # Leave at 50 clusters for now
            ("model", KMeans(n_clusters=4, random_state=0))
        ]
    )
    return pipeline_base

Copy the data frame already encoded

In [ ]:
X = df.copy()
print(X.shape)
X.head(3)

Fit the clustering pipeline

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

## Add cluster predictions to dataset

Add a column "`Clusters`" (with the cluster pipeline predictions) to the transformed data.

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_
print(X.shape)
X.head(3)

In [ ]:
print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# df_analysis is returnded as a numpy array, so we need to convert it back to a DataFrame
df_analysis = pd.DataFrame(df_analysis)

# Set the style for seaborn plots
sns.set_style("whitegrid")

# Create a figure with specified size
plt.figure(figsize=(10, 6))

# Correctly indexing the DataFrame to plot scatter points
sns.scatterplot(
    # Use iloc to access the first column
    x=df_analysis.iloc[:, 0],
    # Use iloc to access the second column
    y=df_analysis.iloc[:, 1],  
    hue=X['Clusters'],
    palette='Set1',
    alpha=0.6
)

# Plot cluster centers
plt.scatter(
    x=pipeline_cluster['model'].cluster_centers_[:, 0], 
    y=pipeline_cluster['model'].cluster_centers_[:, 1],
    marker="x", 
    s=169, 
    linewidths=3, 
    color="black"
)

# Set labels and title for the plot
plt.xlabel("PCA Component 0")
plt.ylabel("PCA Component 1")
plt.title("PCA Components Colored by Clusters")

# Display the plot
plt.show()


Plot a 3D interactive plot for better visualization

In [ ]:
import plotly.express as px
import plotly.graph_objects as go  # Import graph_objects for custom traces
import pandas as pd
import numpy as np

# Ensure 'Clusters' column is added and matches the data correctly
df_analysis["Clusters"] = X["Clusters"]

# Select the first three columns and add 'Clusters' for plotting
# Adjust these column indices if your PCA components are located elsewhere
selected_columns = df_analysis.iloc[
    :, :3
]  # Select first three components (adjust indices if needed)
selected_columns.columns = [
    "PCA Component 0",
    "PCA Component 1",
    "PCA Component 2",
]  # Rename selected columns
selected_columns["Clusters"] = df_analysis["Clusters"]  # Add the clusters column

# Create an interactive 3D scatter plot with Plotly
fig = px.scatter_3d(
    selected_columns,
    x="PCA Component 0",  # Reference by column name
    y="PCA Component 1",  # Reference by column name
    z="PCA Component 2",  # Reference by column name
    color="Clusters",  # Color by cluster labels
    title='Interactive 3D PCA Components Colored by Clusters',
    color_continuous_scale='Viridis',
    opacity=0.7,
)

# Update the trace for data points to adjust marker size
fig.update_traces(marker=dict(size=4))  # Adjust size to desired level

# Extract cluster centers from the clustering model for plotting
cluster_centers = pipeline_cluster["model"].cluster_centers_

# Add cluster centers to the plot
fig.add_trace(
    go.Scatter3d(
        x=cluster_centers[:, 0],
        y=cluster_centers[:, 1],
        z=cluster_centers[:, 2],
        mode="markers",
        # Adjust size here to change cluster centers size
        marker=dict(size=10, color="black", symbol="x"),
        name="Cluster Centers",
    )
)
# Update layout to improve figure size, legend, and color bar placement
fig.update_layout(
    scene=dict(
        xaxis_title='PCA Component 0',
        yaxis_title='PCA Component 1',
        zaxis_title='PCA Component 2',
    ),
    width=1000,  
    height=800,  
    legend=dict(
        orientation="h",  # Horizontal orientation for the legend
        yanchor="top",
        y=1.05,
        xanchor="center",
        x=0.5
    ),
    coloraxis_colorbar=dict(
        title="Clusters",
        x=1.05,  # Adjust the horizontal position of the color bar
        thickness=20,  # Adjust the thickness of the color bar
    )
)
# Show the plot
fig.show()




Save the cluster predictions from this pipeline to use in the future.

In [ ]:
cluster_predictions_with_all_variables = X['Clusters']
cluster_predictions_with_all_variables

## Fit a classifier, where the target is cluster predictions and features remaining variables

Copy `X` to a DataFrame `df_clf`

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df_clf.drop(['Clusters'], axis=1),
    df_clf['Clusters'],
    test_size=0.2,
    random_state=0
)

print(X_train.shape, X_test.shape)


### Create classifier pipeline steps

Use GradientBoostingClassifier as the algorithm since it typically has good performance while being fast to train

In [ ]:
# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithm
from sklearn.ensemble import GradientBoostingClassifier


def PipelineClf2ExplainClusters():
    pipeline_base = Pipeline(
        [
            ("scaler", StandardScaler()),
            (
            "feat_selection",
            SelectFromModel(GradientBoostingClassifier(random_state=0)),
            ),
            ("model", GradientBoostingClassifier(random_state=0)),
        ]
    )

    return pipeline_base


PipelineClf2ExplainClusters()

Fit the classifier to the training data

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train, y_train)

## Evaluate classifier performance on Train and Test Sets

Print classification report for the training set

In [ ]:
from sklearn.metrics import classification_report

print("Training Set Classification Report:")
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

Print classification report for the test set

In [ ]:
print("Test Set Classification Report:")
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

## Assess the most important Features that define a cluster

In [ ]:
columns_after_data_cleaning_feat_eng = df.columns

best_features = columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support(
)].to_list()

# Create DataFrame to display feature importance from the model
df_feature_importance = pd.DataFrame(
    {
        "Feature": best_features,
        "Importance": pipeline_clf_cluster["model"].feature_importances_,
    }
).sort_values(by="Importance", ascending=False)

# Reassign best features in order of importance
best_features = df_feature_importance["Feature"].to_list()

# Display the most important features and plot their importance
print(
    f"* These are the {len(best_features)} most important features in descending order. "
    f"The model was trained on them: \n{best_features} \n"
)

# Plotting feature importances
df_feature_importance.plot(kind="bar", x="Feature", y="Importance")
plt.title("Feature Importance")
plt.ylabel("Importance")
plt.xlabel("Features")
plt.xticks(rotation=90)
plt.show()


Store the best_features to use at a later stage.

In [ ]:
best_features_pipeline_all_variables = best_features
best_features_pipeline_all_variables

## Cluster Analysis

Load function that plots a table with description for all Clusters

In [ ]:

def DescriptionAllClusters(df, decimal_points=3):

    DescriptionAllClusters = pd.DataFrame(
        columns=df.drop(['Clusters'], axis=1).columns)
    # iterate on each cluster , calls Clusters_IndividualDescription()
    for cluster in df.sort_values(by='Clusters')['Clusters'].unique():

        EDA_ClusterSubset = df.query(
            f"Clusters == {cluster}").drop(['Clusters'], axis=1)
        ClusterDescription = Clusters_IndividualDescription(
            EDA_ClusterSubset, cluster, decimal_points)
        DescriptionAllClusters = DescriptionAllClusters.append(
            ClusterDescription)

    DescriptionAllClusters.set_index(['Cluster'], inplace=True)
    return DescriptionAllClusters


def Clusters_IndividualDescription(EDA_Cluster, cluster, decimal_points):

    ClustersDescription = pd.DataFrame(columns=EDA_Cluster.columns)
    # for a given cluster, iterate over all columns
    # if the variable is numerical, calculate the IQR: display as Q1 -- Q3.
    # That will show the range for the most common values for the numerical variable
    # if the variable is categorical, count the frequencies and displays the top 3 most frequent
    # That will show the most common levels for the category

    for col in EDA_Cluster.columns:

        try:  # eventually a given cluster will have only missing data for a given variable

            if EDA_Cluster[col].dtypes == 'object':

                top_frequencies = EDA_Cluster.dropna(
                    subset=[col])[[col]].value_counts(normalize=True).nlargest(n=3)
                Description = ''

                for x in range(len(top_frequencies)):
                    freq = top_frequencies.iloc[x]
                    category = top_frequencies.index[x][0]
                    CategoryPercentage = int(round(freq*100, 0))
                    statement = f"'{category}': {CategoryPercentage}% , "
                    Description = Description + statement

                ClustersDescription.at[0, col] = Description[:-2]

            elif EDA_Cluster[col].dtypes in ['float', 'int']:
                DescStats = EDA_Cluster.dropna(subset=[col])[[col]].describe()
                Q1 = round(DescStats.iloc[4, 0], decimal_points)
                Q3 = round(DescStats.iloc[6, 0], decimal_points)
                Description = f"{Q1} -- {Q3}"
                ClustersDescription.at[0, col] = Description

        except Exception as e:
            ClustersDescription.at[0, col] = 'Not available'
            print(
                f"** Error Exception: {e} - cluster {cluster}, variable {col}")

    ClustersDescription['Cluster'] = str(cluster)

    return ClustersDescription


Load a custom function to plot cluster distribution per Variable (absolute and relative levels)

In [ ]:
import plotly.express as px


def cluster_distribution_per_variable(df, target):
    """
    The data should have 2 variables, the cluster predictions and
    the variable you want to analyze with, in this case we call "target".
    We use plotly express to create 2 plots:
    Cluster distribution across the target.
    Relative presence of the target level in each cluster.
    """
    df_bar_plot = df.value_counts(["Clusters", target]).reset_index()
    df_bar_plot.columns = ["Clusters", target, "Count"]
    df_bar_plot[target] = df_bar_plot[target].astype("object")

    print(f"Clusters distribution across {target} levels")
    fig = px.bar(
        df_bar_plot, x="Clusters", y="Count", color=target, width=800, height=500
    )
    fig.update_layout(xaxis=dict(tickmode="array", tickvals=df["Clusters"].unique()))
    fig.show(renderer="jupyterlab")

    df_relative = (
        df.groupby(["Clusters", target])
        .size()
        .groupby(level=0)
        .apply(lambda x: 100 * x / x.sum())
        .reset_index()
        .sort_values(by=["Clusters"])
    )
    df_relative.columns = ["Clusters", target, "Relative Percentage (%)"]

    print(f"Relative Percentage (%) of {target} in each cluster")
    fig = px.line(
        df_relative,
        x="Clusters",
        y="Relative Percentage (%)",
        color=target,
        width=800,
        height=500,
    )
    fig.update_layout(xaxis=dict(tickmode="array", tickvals=df["Clusters"].unique()))
    fig.update_traces(mode="markers+lines")
    fig.show(renderer="jupyterlab")

Create a DataFrame that contains best features and Clusters Predictions since we want to analyse the patterns for each cluster.


In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
print(df_cluster_profile.shape)
df_cluster_profile.head(3)

We want also to analyse Live birth occurrence levels.

In [ ]:
df_success = pd.read_csv("outputs/datasets/cleaned/FertilityTreatmentDataCleaned.csv").filter(['Live birth occurrence'])
df_success['Live birth occurrence'] = df_success['Live birth occurrence'].astype('object')
df_success.head(3)

### Cluster profile based on the best features

In [ ]:

# Set display options to show the full content of each column
pd.set_option('display.max_colwidth', None)

# Combine the DataFrames and ensure proper alignment
# Make sure df_cluster_profile and df_success have compatible indices or columns
combined_df = pd.concat([df_cluster_profile, df_success], axis=1)

# Check the first few rows to ensure proper concatenation
print(combined_df.head())

# Ensure that DescriptionAllClusters is defined properly and can handle the combined DataFrame
# Provide the combined DataFrame to the description function
clusters_profile = DescriptionAllClusters(df=combined_df, decimal_points=0)

# Display the clusters profile
clusters_profile


### Clusters distribution across Live birth occurrence levels & Relative Percentage of Live birth occurrence in each cluster

In [ ]:
df_cluster_vs_success =  df_success.copy()
df_cluster_vs_success['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_success, target='Live birth occurrence')

# Fit New Cluster Pipeline with most important features

In order to reduce feature space, we will study the trade-off between the previous Cluster Pipeline (fitted with all variables) and Pipeline using the variables that are most important to define the clusters from the previous pipeline

In [ ]:
best_features_pipeline_all_variables

#### Reassign best features names

One hot encoding create new columns and appends the value after the name as `_<value>`.
Therefore it is necessary to rename the features to the original name, so when the pipeline is excecuted, it will find the correct columns on the dataframe.

In [ ]:
import re

def reassign_best_features(features):
    """
    Reassigns encoded feature names to their original names by removing any suffixes added by one-hot encoding.
    """
    # Use regex to remove the "_<value>" suffix from feature names
    reassigned_best_features = [re.sub(r'_[^_]+$', '', feature) for feature in features]
    # Filter out duplicate values from the reassigned features list
    unique_reassigned_best_features = list(set(reassigned_best_features))

    return unique_reassigned_best_features

reassigned_best_features = reassign_best_features(best_features)
reassigned_best_features


## Define trade-off and metrics to compare new and previous Cluster Pipeline

To evaluate this trade-off:
1. Conduct a elbow method and silhouette analysis and check if the same number of clusters is suggested
2. Fit new cluster pipeline and compare if the predictions from this pipeline are "equivalent" to the predictions from the previous pipeline
3. Fit a classifier to explain cluster, and check if performance on Train and Test sets is similar to the previous pipeline
4. Check if the most important features for the classifier are the same from the previous pipeline
5. Compare if the cluster profile from both pipelines are "equivalent"

If yes, a cluster pipeline using the features that best define the clusters from previous pipeline can be used.
* In real-time fewer variables are needed for predicting clusters for the prospect.

## Subset data with the most relevant variables

Reload data:

In [ ]:
df = pd.read_csv("outputs/datasets/cleaned/FertilityTreatmentDataCleaned.csv")

In [ ]:
df_reduced = df.filter(reassigned_best_features)
df_reduced.head(3)

## Rewrite Cluster Pipeline

Update the Feature Engineering Pipeline, considering only the most important variables from the previous pipeline: `['Date of embryo transfer',
 'Embryos transferred from eggs micro-injected',
 'Total embryos thawed',
 'Patient age at treatment']`

In [ ]:
# Define the natural order for each feature
categories = [
    ['18-34', '35-37', '38-39', '40-42', '43-44', '45-50'],  # Patient age at treatment
    ['0 - fresh cycle', '0 - frozen cycle', '1-5', '6-10', '>10']  # Total embryos thawed
]

# Define columns that need ordinal encoding
ordinal_columns = [
    'Patient age at treatment',
    'Total embryos thawed'
]


def PipelineFECluster():
    pipeline_base = Pipeline(
        [   # Use only the best features
            ('ordinal_encoding', OrdinalEncoderWithCategories(
                categories=categories,
                columns=ordinal_columns
                )
            ),
            (
                "one_hot_encoding",
                OneHotEncoder(
                    variables=[
                        "Date of embryo transfer"
                    ],
                ),
            ),
            # it doesn't need SmartCorrelationSelection
        ]
    )

    return pipeline_base

PipelineFECluster()


## Apply Elbow Method and Silhouette analysis

In [ ]:
pipeline_analysis = PipelineFECluster()
df_analysis = pipeline_analysis.fit_transform(df_reduced)

print(df_analysis.shape,'\n', type(df_analysis))

Elbow Analysis

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11))
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

n_cluster_start, n_cluster_stop = 2, 6

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters, random_state=0),
                                      colors='yellowbrick')
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")


## Fit New Cluster Pipeline

We set X as our training set for the cluster. It is a copy of df_reduced already passed through feature engineering processes (already encoded).

In [ ]:
X = df_reduced.copy()
print(X.shape)
X.head(3)

Fit Cluster pipeline

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

## Add cluster predictions to dataset

We add a column "`Clusters`" (with the cluster pipeline predictions) to the dataset

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_
print(X.shape)
X.head(3)

In [ ]:
print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

## Compare current cluster predictions to previous cluster predictions

We just fitted a new cluster pipeline and want to compare if its predictions are "equivalent" to the previous cluster.

These are the predictions from the **previous** cluster pipeline - trained with all variables 

In [ ]:
cluster_predictions_with_all_variables

And these are the predictions from **current** cluster pipeline

In [ ]:
cluster_predictions_with_best_features = X['Clusters'] 
cluster_predictions_with_best_features

We use a confusion matrix to evaluate if the predictions of both pipelines are **"equivalent"**

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(cluster_predictions_with_all_variables, cluster_predictions_with_best_features))

## Fit a classifier, where the target is cluster predictions and features remaining variables

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_clf.drop(['Clusters'], axis=1),
    df_clf['Clusters'],
    test_size=0.2,
    random_state=0
)

print(X_train.shape, X_test.shape)


Rewrite pipeline to explain clusters

In [ ]:
def PipelineClf2ExplainClusters():
    pipeline_base = Pipeline(
        [
            ("scaler", StandardScaler()),
            # no need to consider feature selection step, since we know which features to consider
            ("model", GradientBoostingClassifier(random_state=0)),
        ]
    )
    return pipeline_base


PipelineClf2ExplainClusters()

## Fit a classifier, where the target is cluster labels and features remaining variables

Create and fit a classifier pipeline to learn the feature importance when defining a cluster

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train,y_train)

## Evaluate classifier performance on Train and Test Sets

Print classification report for the training set

In [ ]:
from sklearn.metrics import classification_report

print("Training Set Classification Report:")
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

Print classification report for the test set

In [ ]:
print("Test Set Classification Report:")
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

## Assess Most Important Features

In [ ]:
# since we don't have feature selection step in this pipeline, best_features is Xtrain columns
best_features = X_train.columns.to_list()

# create a DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': best_features,
    'Importance': pipeline_clf_cluster['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

# Plotting feature importances
df_feature_importance.plot(kind="bar", x="Feature", y="Importance")
plt.title("Feature Importance")
plt.ylabel("Importance")
plt.xlabel("Features")
plt.xticks(rotation=90)
plt.show()


## Cluster Analysis

Create a DataFrame that contains the best features and Clusters Predictions: we want to analyse the patterns for each cluster.


In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
print(df_cluster_profile.shape)
df_cluster_profile.head(3)

We want also to analyse Live birth occurence levels

In [ ]:
df_success = pd.read_csv("outputs/datasets/cleaned/FertilityTreatmentDataCleaned.csv").filter(['Live birth occurrence'])
df_success['Live birth occurrence'] = df_success['Live birth occurrence'].astype('object')
df_success.head(3)

### Cluster profile on most important features

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(df= pd.concat([df_cluster_profile,df_success], axis=1), decimal_points=0)
clusters_profile

### Clusters distribution across Live birth occurrence & Relative Percentage of Live birth occurrence in each cluster

In [ ]:
df_cluster_vs_success=  df_success.copy()
df_cluster_vs_success['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_success, target='Live birth occurrence')

## Which pipeline to deploy?

Recap the criteria we consider to evaluate the **trade-off**
1. Conduct an elbow method and silhouette analysis and check if the same number of clusters is suggested.
2. Fit a new cluster pipeline and compare if the predictions from this pipeline are "equivalent" to the predictions from the previous pipeline.
3. Fit a classifier to explain cluster and check if performance on Train and Test sets is similar to the previous pipeline.
4. Check if the most important features for the classifier are the same from the previous pipeline.
5. Compare if the cluster profile from both pipelines is "equivalent".


In [ ]:
pipeline_cluster

# Push files to Repo


We will generate the following files

* Cluster Pipeline
* Train Set
* Feature importance plot
* Clusters Description
* Cluster Silhouette


In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/cluster_analysis/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)


## Cluster pipeline

In [ ]:
pipeline_cluster

In [ ]:
joblib.dump(value=pipeline_cluster, filename=f"{file_path}/cluster_pipeline.pkl")

## Train Set

In [ ]:
print(df_reduced.shape)
df_reduced.head(3)

In [ ]:
df_reduced.to_csv(f"{file_path}/TrainSet.csv", index=False)

## Most important features plot

These are the features that define a cluster

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance', figsize=(8,4))
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance', figsize=(8,4))
plt.savefig(f"{file_path}/features_define_cluster.png", bbox_inches='tight', dpi=150)

## Cluster Profile

In [ ]:
clusters_profile

In [ ]:
clusters_profile.to_csv(f"{file_path}/clusters_profile.csv")

## Cluster silhouette plot

In [ ]:
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(7,5))
fig = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick', ax=axes)
fig.fit(df_analysis)

plt.savefig(f"{file_path}/clusters_silhouette.png", bbox_inches='tight',dpi=150)